#### Let's pytorch-ify the code in 04

In [1]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn

In [2]:
with open('names.txt') as f:
    data = f.read().split('\n')
print(len(data))
print(data[:10])

32033
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [3]:
characters = list(set([i for i in ''.join(data)]))
characters.insert(1, '.')
hmap = {}
for n in range(0,27):
    hmap[sorted(characters)[n]] = n
rev_hmap = {value:key for key,value in hmap.items()}
print(hmap)
print(rev_hmap)

{'.': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '.', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
n = 4
X, y = [], []

for word in data:
    w = '.' * n + word + '.' * n
    # print(w)
    for i in range(len(word)+n-2):
        temp = []
        for j in w[i:i+n]:
            temp.append(hmap[j])
        X.append(temp)
        y.append(hmap[w[i+n]])
        if w[i+n] == '.': break

# for i,j in zip(X,y):
#     print(i,j)
X = torch.tensor(X)   
y = torch.tensor(y)   

In [5]:
X.shape, y.shape

(torch.Size([228146, 4]), torch.Size([228146]))

In [6]:
C = torch.randn((27,2)) # We embed 27 features(letters) into a 2 dimensional space. Will train over this later
emb = C[X]
emb.shape

torch.Size([228146, 4, 2])

In [7]:
h1 = 300
num_epoch = 10000
lr = 0.01

In [8]:
model = nn.Sequential(
    nn.Embedding(27, 2),
     nn.Flatten(),
     nn.Linear(C[X].shape[1]*C[X].shape[2],h1, bias=False), 
     nn.BatchNorm1d(h1), 
     nn.Tanh(),
     nn.Linear(h1,27),
    
)

In [9]:
# Apply Kaiming initialization (He initialization) to layers
for layer in model:
    if isinstance(layer, nn.Linear):
        nn.init.kaiming_normal_(layer.weight, nonlinearity='tanh')

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [11]:
for i in range(num_epoch):
    
    batch = torch.randint(1,len(X),(64,))
    x = X[batch]
    y_ = y[batch]

    logits = model(x)

    loss = F.cross_entropy(logits, y_)
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()

    if  i == 2000 or i == 5000:
        lr = lr/10
     

    if not i%1000: print(f'Epoch {i}: {loss.item():.4f}')

Epoch 0: 3.7702
Epoch 1000: 2.6773
Epoch 2000: 2.6373
Epoch 3000: 2.4755
Epoch 4000: 2.5256
Epoch 5000: 2.4981
Epoch 6000: 2.4844
Epoch 7000: 2.6522
Epoch 8000: 2.5469
Epoch 9000: 2.4863
